In [10]:
import numpy as np
import pandas as pd
from gekko import GEKKO

# -----------------------------
# Load Input Data from CSV
# -----------------------------
df = pd.read_csv('price_promotion_input.csv')  # Columns: item_id, p, c, b, d_max, d_min, d_0, D_0, pe
S = pd.read_csv('substitution_matrix.csv', header=None).values  # Convert to numpy array, skip index column

n_items = df.shape[0]

In [11]:
df

,item,p,c,b,d_max,d_min,d_0,D_0,pe
0,Egg,84.289922,32.533380,0.419669,0.255471,0.01,0.272621,54.714378,2.614880
1,Butter,190.882147,11.277224,1.091318,0.208540,0.01,0.224660,145.461562,2.792183
2,Cheese,150.418879,18.146509,0.261899,0.431495,0.01,0.166180,97.153397,1.636007
3,Juice,125.751820,21.486283,1.836777,0.242701,0.01,0.112712,126.285604,1.220104
4,Cereal,43.863448,25.523149,0.665804,0.212374,0.01,0.162196,186.134971,1.455870
5,Yogurt,43.858986,40.332918,1.392540,0.317078,0.01,0.165037,87.393834,1.854216
6,Bread,25.745468,13.985320,0.761080,0.156370,0.01,0.245921,111.557439,2.636030
7,Fruit,175.242587,28.140550,1.136122,0.420879,0.01,0.227511,163.332671,2.721461
8,Noodles,126.206277,31.658656,1.184079,0.129820,0.01,0.277443,84.319725,1.013904
9,Rice,145.993427,7.090269,0.532738,0.494755,0.01,0.194443,61.546986,2.021495


In [8]:
sub_matrix = pd.read_csv('substitution_matrix.csv')
sub_matrix

,0.0,-0.1489306225047815,-0.0016508964645687296,-0.11947301410993819,-0.12909570337351942,-0.2778678315872803,0.0657386003879381,0.0016074139373168927,-0.2691127492500064,-0.13281212145803314,0.24495953157999223,-0.15626286559981653,-0.21306307674526614,-0.006328343833462202,0.29139027246636046,-0.15476683709309974,0.10328132844352711,0.15697176919723055,-0.15741747360456018,0.13692980916711572
0,-0.079330,0.000000,0.080118,0.021465,-0.245826,0.201181,-0.107532,-0.188089,-0.275535,0.054536,0.106539,-0.290047,0.007256,-0.164103,0.087104,-0.195380,0.114563,-0.067959,0.262038,-0.217487
1,-0.095360,-0.231916,0.000000,0.226404,-0.145235,0.095990,0.190333,0.033120,0.017790,-0.154889,-0.244138,0.238329,0.240251,0.079861,-0.096582,-0.090474,0.135573,0.238266,0.232252,0.167925
2,0.085219,-0.249516,-0.203023,0.000000,0.063857,-0.294482,-0.239117,0.098101,-0.296963,-0.203515,0.029240,0.115137,0.091177,-0.165438,0.127308,-0.157651,-0.104760,0.147895,0.089780,0.209534
3,0.094568,0.040985,-0.243795,-0.079371,0.000000,-0.153606,0.283806,-0.064141,0.235228,0.078683,0.176887,0.001582,0.046142,-0.004489,-0.182854,0.133471,-0.131537,-0.285410,0.087283,-0.193734
4,0.264275,0.272357,0.248919,-0.077905,-0.290726,0.000000,-0.043090,0.279993,0.278172,0.211806,-0.123331,-0.068941,0.210682,-0.109847,-0.198304,0.034081,0.261693,0.117618,0.042037,-0.241694
5,0.069004,0.294032,-0.215950,0.010998,0.226424,0.144461,0.000000,0.121490,-0.084305,-0.123845,0.185617,0.186068,0.220243,0.247944,0.006805,0.000910,0.178977,0.089978,0.121180,0.177476
6,0.234003,-0.097203,-0.074650,-0.243611,0.046968,-0.278435,-0.020641,0.000000,-0.128075,0.054500,-0.281700,-0.277591,0.193560,-0.083886,-0.223764,0.013346,0.161996,-0.170507,0.073734,-0.248792
7,-0.268991,0.018813,0.024381,0.082458,0.135655,0.285511,0.009780,-0.106226,0.000000,-0.137501,-0.036617,-0.252926,-0.284790,0.277589,0.201588,0.117585,-0.054628,-0.196023,-0.206138,-0.149854
8,0.029536,0.128758,0.096118,-0.132040,0.272919,0.142738,0.032612,0.067032,-0.048240,0.000000,-0.086416,0.154708,-0.291364,-0.230356,-0.272398,-0.275563,0.213276,0.122195,-0.015496,-0.241300
9,-0.005030,-0.015917,-0.196079,-0.039689,-0.060897,0.069510,0.081056,-0.272818,-0.075232,0.075516,0.000000,0.213894,0.095216,-0.202239,-0.257659,0.085452,-0.284093,0.051465,0.264138,0.045285


In [14]:
# -----------------------------
# Phase 1: Greedy Heuristic
# -----------------------------
def greedy_selection(df, S, B):
    selected = []
    total_cost = 0
    d_star = []
    net_gains = []

    for i, row in df.iterrows():
        d_vals = np.linspace(row['d_min'], row['d_max'], 50)
        best_gain = -np.inf
        best_d = row['d_min']
        best_cost = 0

        for d in d_vals:
            D_hat = row['D_0'] * ((1 - d) / (1 - row['d_0']))**row['pe']
            delta_D = D_hat - row['D_0']
            own_profit = D_hat * (row['p'] * (1 - d) - row['c'])
            cannibal_loss = sum(S[i, j] * delta_D * (df.loc[j, 'p'] - df.loc[j, 'c']) for j in range(n_items) if j != i)
            net_gain = own_profit - cannibal_loss
            cost = row['b'] * D_hat

            if net_gain / cost > best_gain / cost and (total_cost + cost <= B):
                best_gain = net_gain
                best_d = d
                best_cost = cost

        if total_cost + best_cost <= B:
            selected.append(i)
            total_cost += best_cost
            d_star.append(best_d)
            net_gains.append(best_gain)
        else:
            d_star.append(0)
            net_gains.append(0)

    df['selected'] = [1 if i in selected else 0 for i in range(n_items)]
    df['d_star'] = d_star
    df['net_gain_greedy'] = net_gains
    return df

df = greedy_selection(df, S, B=1500)

# -----------------------------
# Phase 2: MILP Approximation via Gekko
# -----------------------------
m = GEKKO(remote=False)
m.options.SOLVER = 1  # APOPT

delta = [m.Var(value=int(df.loc[i, 'selected']), lb=0, ub=1, integer=True) for i in range(n_items)]
d = [m.Var(value=df.loc[i, 'd_star'], lb=0, ub=df.loc[i, 'd_max']) for i in range(n_items)]

# Apply min discount constraints for selected items
for i in range(n_items):
    m.Equation(d[i] >= delta[i] * df.loc[i, 'd_star'])
    m.Equation(d[i] <= delta[i] * df.loc[i, 'd_max'])

net_gain_expr = []
budget_expr = []

for i in range(n_items):
    D_0 = df.loc[i, 'D_0']
    d_0 = df.loc[i, 'd_0']
    pe_i = df.loc[i, 'pe']
    p_i = df.loc[i, 'p']
    c_i = df.loc[i, 'c']
    b_i = df.loc[i, 'b']
    d_i0 = df.loc[i, 'd_star']

    D_hat_0 = D_0 * ((1 - d_i0) / (1 - d_0))**pe_i
    dD_dd = -pe_i * D_hat_0 / (1 - d_i0)
    D_hat_lin = D_hat_0 + dD_dd * (d[i] - d_i0)

    own_profit_0 = D_hat_0 * (p_i * (1 - d_i0) - c_i)
    slope_own = (p_i * (1 - d_i0) - c_i) * dD_dd - D_hat_0 * p_i

    cannibal_0 = sum(S[i, j] * (D_hat_0 - D_0) * (df.loc[j, 'p'] - df.loc[j, 'c']) for j in range(n_items) if j != i)
    slope_cannibal = sum(S[i, j] * (df.loc[j, 'p'] - df.loc[j, 'c']) * dD_dd for j in range(n_items) if j != i)

    net_gain_i = own_profit_0 - cannibal_0 + (slope_own - slope_cannibal) * (d[i] - d_i0)
    net_gain_expr.append(delta[i] * net_gain_i)

    budget_expr.append(delta[i] * b_i * D_hat_lin)

m.Maximize(m.sum(net_gain_expr))
m.Equation(m.sum(budget_expr) <= 1500)
m.solve(disp=True)

results = pd.DataFrame({
    'item': df['item'],
    'delta_final': [int(delta[i].value[0]) for i in range(n_items)],
    'd_final': [round(d[i].value[0], 4) for i in range(n_items)],
    'd_greedy': df['d_star'],
    'selected_greedy': df['selected']
})

print(results)

 ----------------------------------------------------------------
 APMonitor, Version 1.0.3
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :  2
   Constants    :  0
   Variables    :  123
   Intermediates:  0
   Connections  :  42
   Equations    :  82
   Residuals    :  82
 
 Number of state variables:    123
 Number of total equations: -  83
 Number of slack variables: -  41
 ---------------------------------------
 Degrees of freedom       :    -1
 
 * Warning: DOF <= 0
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.01 NLPi:    8 Dpth:    0 Lvs:    2 Obj: -2.40E+05 Gap:       NaN
Iter:     2 I:  0 Tm:      0.00 NLPi:    5 Dpth:    1 Lvs:    3 Obj: -2.38E+05 Gap:       NaN
Iter:     3 I:  0 Tm:      0.01 NLPi:   15 Dpth:    1 Lvs:  